In [1]:
from onnx import TensorProto
from onnx.helper import (
    make_model, make_node, make_graph,
    make_tensor_value_info)
from onnx.checker import check_model


In [3]:
# 'X' is the name, TensorProto.FLOAT the type, [None, None] the shape
X = make_tensor_value_info('X', TensorProto.FLOAT, ["batch_size", "feature_dim"])
A = make_tensor_value_info('A', TensorProto.FLOAT, ["feature_dim"])
B = make_tensor_value_info('B', TensorProto.FLOAT, [])

# outputs, the shape is left undefined
Y = make_tensor_value_info('Y', TensorProto.FLOAT, ["batch_size"])

# nodes

# It creates a node defined by the operator type MatMul,
# 'X', 'A' are the inputs of the node, 'XA' the output.
node1 = make_node('MatMul', ['X', 'A'], ['XA'], 'XA')
node2 = make_node('Add', ['XA', 'B'], ['Y'], 'Y')

# from nodes to graph
# the graph is built from the list of nodes, the list of inputs,
# the list of outputs and a name.

graph = make_graph([node1, node2],  # nodes
                    'lr',  # a name
                    [X, A, B],  # inputs
                    [Y])  # outputs

# onnx graph
# there is no metadata in this case.

onnx_model = make_model(graph)

# Let's check the model is consistent,
# this function is described in section
# Checker and Shape Inference.
check_model(onnx_model)

In [4]:
with open("linear_regression.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [6]:
import onnxruntime as ort
import numpy as np

x = np.array([[1.0, 2.0, 3.0], [100.0, 101.0, 102.0]], dtype=np.float32)
a = np.array([1.0, 0.1, 0.01], dtype=np.float32)
b = np.array(1000.0, dtype=np.float32)

ort_sess = ort.InferenceSession('linear_regression.onnx')
outputs = ort_sess.run(['Y'], {'X': x, 'A': a, 'B':b})
print(outputs[0])

[1001.23 1111.12]
